In [ ]:
from pydicom import dcmread  
import os
import SimpleITK as sitk

In [ ]:
def dicom_hhmmss(t):   
    t = str(t)
    if len(t) == 5:    
     	t = '0'+t
    h_t = float(t[0:2])
    m_t = float(t[2:4])
    s_t = float(t[4:6])
    return h_t*3600+m_t*60+s_t


In [ ]:
def PET2SUV(data, PET,norm):
    
    [ST, AT, PW, RST, RTD, RHL, RS, RI] = data 		
    decay_time = dicom_hhmmss(ST)-dicom_hhmmss(RST)
    decay_dose = float(RTD) * pow(2, -float(decay_time) / float(RHL))
    SUVbwScaleFactor = (1000 * float(PW)) / decay_dose
    if norm:
        PET_SUV = (PET * float(RS) + float(RI)) * SUVbwScaleFactor    
    else:
        PET_SUV = PET * SUVbwScaleFactor    	
    return PET_SUV


In [ ]:
path = "./PR"
for i in os.listdir(path):
    if i != "0" and i !="1":
        continue
    folder_a = os.listdir(os.path.join(path,i))
    # df_PET_ID = pd.DataFrame([str(folder)], columns =["ID"])
    # label = pd.DataFrame([1],columns=["label"])
    for folder in folder_a:
        imageName =  "PET"
        petName = "PET.nii.gz"
        if i == "0":
            imagePath = os.path.join('./label0',folder, imageName)
            dicom_path = os.path.join(imagePath,os.listdir(imagePath)[0])
        elif i == '1':
            imagePath = os.path.join('./label1',folder, imageName)
            dicom_path = os.path.join(imagePath,os.listdir(imagePath)[0])
        else: continue
        dataDir = os.path.join(path,i, folder)
        dcm = dcmread(dicom_path)
        RadiopharmaceuticalInformationSequence = dcm.RadiopharmaceuticalInformationSequence[0]
        RadiopharmaceuticalStartTime = str(RadiopharmaceuticalInformationSequence['RadiopharmaceuticalStartTime'].value)
        RadionuclideTotalDose = str(RadiopharmaceuticalInformationSequence['RadionuclideTotalDose'].value)
        RadionuclideHalfLife = str(RadiopharmaceuticalInformationSequence['RadionuclideHalfLife'].value)
        dcm_tag = [str(dcm.SeriesTime),str(dcm.AcquisitionTime),str(dcm.PatientWeight),RadiopharmaceuticalStartTime,
                    RadionuclideTotalDose,RadionuclideHalfLife,str(dcm.RescaleSlope),str(dcm.RescaleIntercept)]
        image = sitk.ReadImage(os.path.join(dataDir,petName))
        data = sitk.GetArrayFromImage(image)
        new_pet = PET2SUV(dcm_tag,data,norm=False)
        out = sitk.GetImageFromArray(new_pet)
        sitk.WriteImage(out,os.path.join(dataDir,'SUV.nii.gz'))
        